# Preprocessing<a id='Preprocessing'></a>

## 1 Contents<a id='1_Contents'></a>
* [Preprocessing](#Preprocessing)
  * [1 Contents](#1_Contents)
  * [2 Introduction](#2_Introduction)
  * [3 Imports](#3_Imports)
  * [4 Load the data](#4_Load_the_data)
  * [5 Time snapshots](#5_Time_snapshots)
  * [6 Features](#6_Features)
  * [7 Test train split](#7_Test_train_split)
  * [8 Imputing](#8_Imputing)
  * [9 Save the data](#9_Save_the_data)
  * [10 Summary](#10_Summary)

## 2 Introduction<a id='2_Introduction'></a>

There are a number of tasks to accomplish before we can move on to the modeling step. First of all, the current state of the data is not such that we can simply take a subset of our columns to form our X and another column as y, so we must start by reorganizing the data. We create a new dataframe multi-indexed by time-window and player_id, with columns for the player's stats in that window and in the preceding five windows. Having done this, we move on to creating some more elaborate features, most importantly taking "per game" stats instead of absolute stats during each window.

We split our data into a training set, a test set of *all* of the most recent two windows (9% of the data), and a randomized test set of the older windows (11% of the data). The reasoning for taking separate test sets is that each comes with its own pros/cons: testing with the most recent two windows gives us a glimpse of how a model does with 100% new information in the future, but testing with randomized windows avoids any bias from the particularities of the most recent season.

At the end, we impute missing values and save our data, holding off on normalization until the modeling step in the next notebook.

## 3 Imports<a id='3_Imports'></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import numpy as np

from library.sb_utils import save_file
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

## 4 Load the data<a id='4_Load_the_data'></a>

In [3]:
datapath = '../data/interim/'

names = ['player_data_post_eda', 'team_data_post_eda', 'player_info_post_eda', 'team_info_post_eda', 'windows']

df_player_data = pd.read_csv(datapath + 'player_data_post_eda.csv')
df_player_info = pd.read_csv(datapath + 'player_info_post_eda.csv')
df_team_data = pd.read_csv(datapath + 'team_data_post_eda.csv')
df_team_info = pd.read_csv(datapath + 'team_info_post_eda.csv')
df_windows = pd.read_csv(datapath + 'windows.csv')

## 5 Time snapshots<a id='5_Time_snapshots'></a>

In the previous notebook, we broke up our data into time windows. What we want to do now is to create more elaborate dataframe which contains the stats for each time window, as well as the stats for each of the five preceding windows.

In [6]:
df_player_data.head()

,player_id,attempted_shots_0,shots_on_goal_0,goals_0,assists_0,games_0,toi_0,pp_toi_0,onice_attempts_0,team_id_0,...,team_id_26,attempted_shots_27,shots_on_goal_27,goals_27,assists_27,games_27,toi_27,pp_toi_27,onice_attempts_27,team_id_27
0,8466148,89.0,59.0,8.0,7.0,21.0,25096.0,3337.0,423.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8465058,55.0,16.0,2.0,7.0,20.0,27325.0,4313.0,475.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8476906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,40.0,20.0,NaN,3.0,10.0,10911.0,1597.0,189.0,4.0
3,8466285,48.0,22.0,1.0,7.0,17.0,22531.0,2380.0,356.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8470607,109.0,47.0,2.0,14.0,30.0,44955.0,4695.0,751.0,16.0,...,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
melted_players = df_player_data.melt(id_vars='player_id')

melted_players['window'] = melted_players['variable'].str.split('_').apply(lambda x : x[-1]).astype(int)
melted_players['variable'] = melted_players['variable'].str.split('_').apply(lambda x : '_'.join(x[:-1]))

pivoted_players = melted_players.pivot(index=['window','player_id'], columns='variable', values='value')

In [5]:
pivoted_players.head()

variable          assists  attempted_shots  games  goals  onice_attempts  \
window player_id                                                           
0      8444894        NaN              NaN    NaN    NaN             NaN   
       8444919        NaN              NaN    NaN    NaN             NaN   
       8445000        NaN              NaN    NaN    NaN             NaN   
       8445176        NaN              NaN    NaN    NaN             NaN   
       8445266        NaN              NaN    NaN    NaN             NaN   

variable          pp_toi  shots_on_goal  team_id  toi  
window player_id                                       
0      8444894       NaN            NaN      NaN  NaN  
       8444919       NaN            NaN      NaN  NaN  
       8445000       NaN            NaN      NaN  NaN  
       8445176       NaN            NaN      NaN  NaN  
       8445266       NaN            NaN      NaN  NaN

In [6]:
melted_teams = df_team_data.melt(id_vars='team_id')

melted_teams['window'] = melted_teams['variable'].str.split('_').apply(lambda x : x[-1]).astype(int)
melted_teams['variable'] = melted_teams['variable'].str.split('_').apply(lambda x : '_'.join(x[:-1]))

pivoted_teams = melted_teams.pivot(index=['window','team_id'], columns='variable', values='value')

In [7]:
windows = pd.DataFrame([[5-j+i for j in range(6)] for i in range(23)])

df_player = df_player_info.merge(windows, how='cross')

df_team = df_team_info.merge(windows, how='cross')

In [8]:
df = df_player.merge(pivoted_players, how='inner', left_on=[0, 'player_id'], right_index=True)

for i in range(1,6):
    df = df.merge(pivoted_players, how='inner', left_on=[i, 'player_id'], right_index=True, suffixes=[None, '_' + str(i)])

df = df.drop([1,2,3,4,5], axis=1)

df = df.rename(columns={0:'window'}).set_index(['player_id', 'window'])

df.head()

isForward   birthDate  height_cm  weight  assists  \
player_id window                                                      
8466148   5            True  1979-01-12     185.42   207.0     14.0   
          6            True  1979-01-12     185.42   207.0     10.0   
          7            True  1979-01-12     185.42   207.0      4.0   
          8            True  1979-01-12     185.42   207.0      9.0   
          9            True  1979-01-12     185.42   207.0     14.0   

                  attempted_shots  games  goals  onice_attempts  pp_toi  ...  \
player_id window                                                         ...   
8466148   5                 131.0   27.0    8.0           500.0  5160.0  ...   
          6                 144.0   26.0   11.0           506.0  4407.0  ...   
          7                  61.0   14.0    6.0           231.0  1927.0  ...   
          8                 147.0   23.0   11.0           459.0  3049.0  ...   
          9                 161.0   28.0   12.0           585.0  3396.0  ...   

                    toi_4  assists_5  attempted_shots_5  games_5  goals_5  \
player_id window                                                            
8466148   5       19152.0        7.0               89.0     21.0      8.0   
          6       32608.0        7.0               88.0     17.0      4.0   
          7       32266.0       14.0              126.0     27.0     12.0   
          8       31715.0       15.0              144.0     27.0     11.0   
          9       33047.0       18.0              138.0     27.0      9.0   

                  onice_attempts_5  pp_toi_5  shots_on_goal_5  team_id_5  \
player_id window                                                           
8466148   5                  423.0    3337.0             59.0       16.0   
          6                  345.0    3063.0             53.0       16.0   
          7                  523.0    4131.0             82.0       16.0   
          8                  541.0    5578.0             90.0       16.0   
          9                  516.0    4630.0             75.0       16.0   

                    toi_5  
player_id window           
8466148   5       25096.0  
          6       19152.0  
          7       32608.0  
          8       32266.0  
          9       31715.0  

[5 rows x 58 columns]

In [9]:
dft = df_team.merge(pivoted_teams, how='inner', left_on=[0, 'team_id'], right_index=True)

for i in range(1,6):
    dft = dft.merge(pivoted_teams, how='inner', left_on=[i, 'team_id'], right_index=True, suffixes=[None, '_' + str(i)])

dft = dft.drop([1,2,3,4,5], axis=1)

dft = dft.rename(columns={0:'window'}).set_index(['team_id', 'window'])

In [10]:
dft = dft.drop(columns=['franchiseId', 'shortName', 'teamName', 'abbreviation', 'link'])

In [11]:
dft = dft.drop(columns=['attempts_by_team', 'games_by_team', 'goals_by_team'])

dft.head()

attempts_by_team_1  games_by_team_1  goals_by_team_1  \
team_id window                                                         
1       5                   1239.0             29.0             89.0   
        6                   1290.0             27.0             74.0   
        7                   1367.0             28.0             71.0   
        8                   1023.0             20.0             41.0   
        9                   1192.0             28.0             61.0   

                attempts_by_team_2  games_by_team_2  goals_by_team_2  \
team_id window                                                         
1       5                   1277.0             26.0             65.0   
        6                   1239.0             29.0             89.0   
        7                   1290.0             27.0             74.0   
        8                   1367.0             28.0             71.0   
        9                   1023.0             20.0             41.0   

                attempts_by_team_3  games_by_team_3  goals_by_team_3  \
team_id window                                                         
1       5                   1282.0             27.0             56.0   
        6                   1277.0             26.0             65.0   
        7                   1239.0             29.0             89.0   
        8                   1290.0             27.0             74.0   
        9                   1367.0             28.0             71.0   

                attempts_by_team_4  games_by_team_4  goals_by_team_4  \
team_id window                                                         
1       5                   1404.0             28.0             68.0   
        6                   1282.0             27.0             56.0   
        7                   1277.0             26.0             65.0   
        8                   1239.0             29.0             89.0   
        9                   1290.0             27.0             74.0   

                attempts_by_team_5  games_by_team_5  goals_by_team_5  
team_id window                                                        
1       5                   1315.0             27.0             50.0  
        6                   1404.0             28.0             68.0  
        7                   1282.0             27.0             56.0  
        8                   1277.0             26.0             65.0  
        9                   1239.0             29.0             89.0

In [12]:
def team_per_game(s):
    total_games = dft['games_by_team_1'] + dft['games_by_team_2'] + dft['games_by_team_3'] + dft['games_by_team_4'] + dft['games_by_team_5']
    weighted_sum = dft[s+'s_by_team_1'] * 1.5 + dft[s+'s_by_team_2'] * 1.25 + dft[s+'s_by_team_3'] * 1 + dft[s+'s_by_team_4'] * 0.75 + dft[s+'s_by_team_5'] * 0.5
    return weighted_sum / total_games

In [13]:
dft['weighted_avg_recent_attempts_per_game'] = team_per_game('attempt')
dft['weighted_avg_recent_goals_per_game'] = team_per_game('goal')

dft.head()

attempts_by_team_1  games_by_team_1  goals_by_team_1  \
team_id window                                                         
1       5                   1239.0             29.0             89.0   
        6                   1290.0             27.0             74.0   
        7                   1367.0             28.0             71.0   
        8                   1023.0             20.0             41.0   
        9                   1192.0             28.0             61.0   

                attempts_by_team_2  games_by_team_2  goals_by_team_2  \
team_id window                                                         
1       5                   1277.0             26.0             65.0   
        6                   1239.0             29.0             89.0   
        7                   1290.0             27.0             74.0   
        8                   1367.0             28.0             71.0   
        9                   1023.0             20.0             41.0   

                attempts_by_team_3  games_by_team_3  goals_by_team_3  \
team_id window                                                         
1       5                   1282.0             27.0             56.0   
        6                   1277.0             26.0             65.0   
        7                   1239.0             29.0             89.0   
        8                   1290.0             27.0             74.0   
        9                   1367.0             28.0             71.0   

                attempts_by_team_4  games_by_team_4  goals_by_team_4  \
team_id window                                                         
1       5                   1404.0             28.0             68.0   
        6                   1282.0             27.0             56.0   
        7                   1277.0             26.0             65.0   
        8                   1239.0             29.0             89.0   
        9                   1290.0             27.0             74.0   

                attempts_by_team_5  games_by_team_5  goals_by_team_5  \
team_id window                                                         
1       5                   1315.0             27.0             50.0   
        6                   1404.0             28.0             68.0   
        7                   1282.0             27.0             56.0   
        8                   1277.0             26.0             65.0   
        9                   1239.0             29.0             89.0   

                weighted_avg_recent_attempts_per_game  \
team_id window                                          
1       5                                   47.060219   
        6                                   46.892336   
        7                                   47.450730   
        8                                   46.930769   
        9                                   45.611742   

                weighted_avg_recent_goals_per_game  
team_id window                                      
1       5                                 2.531022  
        6                                 2.651460  
        7                                 2.662409  
        8                                 2.488462  
        9                                 2.376894

## 6 Features<a id='6_Features'></a>

In [14]:
df.columns

Index(['isForward', 'birthDate', 'height_cm', 'weight', 'assists',
       'attempted_shots', 'games', 'goals', 'onice_attempts', 'pp_toi',
       'shots_on_goal', 'team_id', 'toi', 'assists_1', 'attempted_shots_1',
       'games_1', 'goals_1', 'onice_attempts_1', 'pp_toi_1', 'shots_on_goal_1',
       'team_id_1', 'toi_1', 'assists_2', 'attempted_shots_2', 'games_2',
       'goals_2', 'onice_attempts_2', 'pp_toi_2', 'shots_on_goal_2',
       'team_id_2', 'toi_2', 'assists_3', 'attempted_shots_3', 'games_3',
       'goals_3', 'onice_attempts_3', 'pp_toi_3', 'shots_on_goal_3',
       'team_id_3', 'toi_3', 'assists_4', 'attempted_shots_4', 'games_4',
       'goals_4', 'onice_attempts_4', 'pp_toi_4', 'shots_on_goal_4',
       'team_id_4', 'toi_4', 'assists_5', 'attempted_shots_5', 'games_5',
       'goals_5', 'onice_attempts_5', 'pp_toi_5', 'shots_on_goal_5',
       'team_id_5', 'toi_5'],
      dtype='object')

Some of the columns above are already features we would like to include in our model: isForward, height_cm, weight. Let's make sure these are numerical datatypes.

In [15]:
df.loc[:,['isForward', 'height_cm', 'weight']].info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 80293 entries, (8466148, 5) to (8477352, 27)
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   isForward  80293 non-null  bool   
 1   height_cm  80293 non-null  float64
 2   weight     80293 non-null  float64
dtypes: bool(1), float64(2)
memory usage: 1.7 MB


In [16]:
df['isForward'] = df['isForward'].astype(int)

For the rest of the features, we combine two or more columns, or use the teams dataframe. One feature we would like to use is age. Since players' production tends to peak mid-way through their career, it also makes sense to include age squared as a feature.

In [17]:
df.reset_index(inplace=True)

In [18]:
df = df.merge(df_windows, how='inner', left_on='window', right_index=True)

In [19]:
df['birthDate']= pd.to_datetime(df['birthDate'])
df['start']= pd.to_datetime(df['start'])

In [20]:
df['age'] = (df['start'] - df['birthDate']).dt.days

In [21]:
df['age_squared'] = df['age'] ** 2

In [22]:
df.head()

,player_id,window,isForward,birthDate,height_cm,weight,assists,attempted_shots,games,goals,...,goals_5,onice_attempts_5,pp_toi_5,shots_on_goal_5,team_id_5,toi_5,start,end,age,age_squared
0,8466148,5,1,1979-01-12,185.42,207.0,14.0,131.0,27.0,8.0,...,8.0,423.0,3337.0,59.0,16.0,25096.0,2012-02-12,2013-01-19,12084,146023056
23,8465058,5,0,1978-09-03,185.42,210.0,5.0,61.0,27.0,1.0,...,2.0,475.0,4313.0,16.0,3.0,27325.0,2012-02-12,2013-01-19,12215,149206225
46,8476906,5,0,1993-04-20,180.34,180.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2012-02-12,2013-01-19,6872,47224384
69,8466285,5,0,1979-05-23,177.80,192.0,15.0,104.0,28.0,1.0,...,1.0,356.0,2380.0,22.0,16.0,22531.0,2012-02-12,2013-01-19,11953,142874209
92,8470607,5,0,1985-04-20,190.50,220.0,8.0,121.0,27.0,4.0,...,2.0,751.0,4695.0,47.0,16.0,44955.0,2012-02-12,2013-01-19,9794,95922436


We would like to replace many of these stats with a per-game version.

In [23]:
colnames = ['assists', 'attempted_shots', 'goals', 'onice_attempts', 'pp_toi',
       'shots_on_goal', 'toi']

# colnames_indexed = [col + '_' + str(i) for col in colnames for i in range(1,6)]

In [24]:
def divide_by_column(df, numerators, denominator):
    for num in numerators:
        df[num + '_per_' + denominator] = df[num] / df[denominator]

In [25]:
divide_by_column(df, colnames, 'games')

for i in range(1,6):
    divide_by_column(df, [col + '_' + str(i) for col in colnames], 'games' + '_' + str(i))

In [26]:
df.columns

Index(['player_id', 'window', 'isForward', 'birthDate', 'height_cm', 'weight',
       'assists', 'attempted_shots', 'games', 'goals',
       ...
       'pp_toi_4_per_games_4', 'shots_on_goal_4_per_games_4',
       'toi_4_per_games_4', 'assists_5_per_games_5',
       'attempted_shots_5_per_games_5', 'goals_5_per_games_5',
       'onice_attempts_5_per_games_5', 'pp_toi_5_per_games_5',
       'shots_on_goal_5_per_games_5', 'toi_5_per_games_5'],
      dtype='object', length=106)

Let's now add in the information about recent performance for each player's team, and then clean up the dataframe.

In [27]:
df = df.merge(dft.reset_index()[['team_id', 'window', 'weighted_avg_recent_attempts_per_game', 'weighted_avg_recent_goals_per_game']], how='left', on=['team_id', 'window'])

Let's rename the columns we just added, drop rows for players that did not play enough games in the given window, and drop the columns we are not going to use as features.

In [28]:
df = df.rename(columns={'weighted_avg_recent_attempts_per_game' : 'recent_team_attempts_per_game', 'weighted_avg_recent_goals_per_game' : 'recent_team_goals_per_game'})

In [29]:
df = df.dropna(subset='games')

In [30]:
df = df[df['games'] >= 10]

In [31]:
df.shape

(14188, 108)

In [32]:
df_final = df[['player_id', 'window', 'assists_per_games','goals_per_games',
       'shots_on_goal_per_games', 'isForward', 'height_cm',
       'weight', 'age', 'age_squared', 'assists_1_per_games_1', 'attempted_shots_1_per_games_1',
       'goals_1_per_games_1', 'onice_attempts_1_per_games_1',
       'pp_toi_1_per_games_1', 'shots_on_goal_1_per_games_1',
       'toi_1_per_games_1', 'assists_2_per_games_2',
       'attempted_shots_2_per_games_2', 'goals_2_per_games_2',
       'onice_attempts_2_per_games_2', 'pp_toi_2_per_games_2',
       'shots_on_goal_2_per_games_2', 'toi_2_per_games_2',
       'assists_3_per_games_3', 'attempted_shots_3_per_games_3',
       'goals_3_per_games_3', 'onice_attempts_3_per_games_3',
       'pp_toi_3_per_games_3', 'shots_on_goal_3_per_games_3',
       'toi_3_per_games_3', 'assists_4_per_games_4',
       'attempted_shots_4_per_games_4', 'goals_4_per_games_4',
       'onice_attempts_4_per_games_4', 'pp_toi_4_per_games_4',
       'shots_on_goal_4_per_games_4', 'toi_4_per_games_4',
       'assists_5_per_games_5', 'attempted_shots_5_per_games_5',
       'goals_5_per_games_5', 'onice_attempts_5_per_games_5',
       'pp_toi_5_per_games_5', 'shots_on_goal_5_per_games_5',
       'toi_5_per_games_5', 'recent_team_attempts_per_game',
       'recent_team_goals_per_game']].set_index(['window', 'player_id'])

df_final.head()

assists_per_games  goals_per_games  shots_on_goal_per_games  \
window player_id                                                                
5      8466148             0.518519         0.296296                 2.629630   
       8465058             0.185185         0.037037                 1.000000   
       8466285             0.535714         0.035714                 1.571429   
       8470607             0.296296         0.148148                 2.111111   
       8474668             0.066667         0.066667                 0.600000   

                  isForward  height_cm  weight    age  age_squared  \
window player_id                                                     
5      8466148            1     185.42   207.0  12084    146023056   
       8465058            0     185.42   210.0  12215    149206225   
       8466285            0     177.80   192.0  11953    142874209   
       8470607            0     190.50   220.0   9794     95922436   
       8474668            1     187.96   206.0   8591     73805281   

                  assists_1_per_games_1  attempted_shots_1_per_games_1  ...  \
window player_id                                                        ...   
5      8466148                 0.666667                       5.111111  ...   
       8465058                 0.269231                       1.846154  ...   
       8466285                 0.481481                       2.629630  ...   
       8470607                 0.370370                       3.962963  ...   
       8474668                 0.038462                       1.038462  ...   

                  toi_4_per_games_4  assists_5_per_games_5  \
window player_id                                             
5      8466148          1126.588235               0.333333   
       8465058          1199.315789               0.350000   
       8466285          1441.083333               0.411765   
       8470607          1432.833333               0.466667   
       8474668           390.300000                    NaN   

                  attempted_shots_5_per_games_5  goals_5_per_games_5  \
window player_id                                                       
5      8466148                         4.238095             0.380952   
       8465058                         2.750000             0.100000   
       8466285                         2.823529             0.058824   
       8470607                         3.633333             0.066667   
       8474668                              NaN                  NaN   

                  onice_attempts_5_per_games_5  pp_toi_5_per_games_5  \
window player_id                                                       
5      8466148                       20.142857            158.904762   
       8465058                       23.750000            215.650000   
       8466285                       20.941176            140.000000   
       8470607                       25.033333            156.500000   
       8474668                             NaN                   NaN   

                  shots_on_goal_5_per_games_5  toi_5_per_games_5  \
window player_id                                                   
5      8466148                       2.809524        1195.047619   
       8465058                       0.800000        1366.250000   
       8466285                       1.294118        1325.352941   
       8470607                       1.566667        1498.500000   
       8474668                            NaN                NaN   

                  recent_team_attempts_per_game  recent_team_goals_per_game  
window player_id                                                             
5      8466148                        54.191606                    3.131387  
       8465058                        54.521898                    2.713504  
       8466285                        52.860294                    2.413603  
       8470607                        54.191606                    3.131387  
       8474668 

## 7 Test train split<a id='7_Test_train_split'></a>

We start by defining X and Y. (For now, we are keeping a Y with multiple columns. In our model-training phase, we can select which column to serve as our y variable.)

In [33]:
Y = df_final[['assists_per_games', 'goals_per_games', 'shots_on_goal_per_games']]
X = df_final.drop(columns=['assists_per_games', 'goals_per_games', 'shots_on_goal_per_games'])

In [34]:
X.shape

(14188, 42)

In [35]:
Y.shape

(14188, 3)

We would like to split up our test data in two ways: in order to see how our model will do on data from the future, we would like to limit our training to all but the last two windows, reserving the last two windows for testing. However, to avoid any bias from those particular windows, we also take a random subset for testing.

In [36]:
X_lastwindows = X.loc[[26, 27]]
X_firstwindows = X.loc[5:25]
Y_lastwindows = Y.loc[[26, 27]]
Y_firstwindows = Y.loc[5:25]

X_lastwindows.shape, Y_lastwindows.shape

((1302, 42), (1302, 3))

In [37]:
1302/14188

0.09176769100648435

We've separated off 9% of our data into Y_lastwindows, so let's separate off another 12% randomly for testing.

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(X_firstwindows, Y_firstwindows, test_size=0.12, random_state=37)

In [39]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((11339, 42), (1547, 42), (11339, 3), (1547, 3))

## 8 Imputing<a id='8_Imputing'></a>

Because there have been new teams added to the league, we are missing team data for a number of entries.

In [40]:
X_train[X_train[['recent_team_attempts_per_game', 'recent_team_goals_per_game']].T.isna().any()].shape[0]

212

In [41]:
X_test[X_test[['recent_team_attempts_per_game', 'recent_team_goals_per_game']].T.isna().any()].shape[0]

42

In [42]:
X_lastwindows[X_lastwindows[['recent_team_attempts_per_game', 'recent_team_goals_per_game']].T.isna().any()].shape[0]

0

Let's fill in these missing values with the league average for the given window.

In [45]:
team_attempts_means = X_train.groupby('window').agg(mean_team_attempts=('recent_team_attempts_per_game', 'mean'))
team_goals_means = X_train.groupby('window').agg(mean_team_goals=('recent_team_goals_per_game', 'mean'))

team_attempts_fill_train = X_train.merge(team_attempts_means, how='inner', left_index=True, right_index=True)['mean_team_attempts']
team_attempts_fill_test = X_test.merge(team_attempts_means, how='inner', left_index=True, right_index=True)['mean_team_attempts']
team_goals_fill_train = X_train.merge(team_goals_means, how='inner', left_index=True, right_index=True)['mean_team_goals']
team_goals_fill_test = X_test.merge(team_goals_means, how='inner', left_index=True, right_index=True)['mean_team_goals']

X_train['recent_team_attempts_per_game'] = X_train['recent_team_attempts_per_game'].fillna(value=team_attempts_fill_train)
X_train['recent_team_goals_per_game'] = X_train['recent_team_goals_per_game'].fillna(value=team_goals_fill_train)
X_test['recent_team_attempts_per_game'] = X_test['recent_team_attempts_per_game'].fillna(value=team_attempts_fill_test)
X_test['recent_team_goals_per_game'] = X_test['recent_team_goals_per_game'].fillna(value=team_goals_fill_test)

In [46]:
X_train[X_train[['recent_team_attempts_per_game', 'recent_team_goals_per_game']].T.isna().any()]

,,isForward,height_cm,weight,age,age_squared,assists_1_per_games_1,attempted_shots_1_per_games_1,goals_1_per_games_1,onice_attempts_1_per_games_1,pp_toi_1_per_games_1,...,toi_4_per_games_4,assists_5_per_games_5,attempted_shots_5_per_games_5,goals_5_per_games_5,onice_attempts_5_per_games_5,pp_toi_5_per_games_5,shots_on_goal_5_per_games_5,toi_5_per_games_5,recent_team_attempts_per_game,recent_team_goals_per_game
window,player_id,,,,,,,,,,,,,,,,,,,,,


In [47]:
X_test[X_test[['recent_team_attempts_per_game', 'recent_team_goals_per_game']].T.isna().any()]

,,isForward,height_cm,weight,age,age_squared,assists_1_per_games_1,attempted_shots_1_per_games_1,goals_1_per_games_1,onice_attempts_1_per_games_1,pp_toi_1_per_games_1,...,toi_4_per_games_4,assists_5_per_games_5,attempted_shots_5_per_games_5,goals_5_per_games_5,onice_attempts_5_per_games_5,pp_toi_5_per_games_5,shots_on_goal_5_per_games_5,toi_5_per_games_5,recent_team_attempts_per_game,recent_team_goals_per_game
window,player_id,,,,,,,,,,,,,,,,,,,,,


Now let's see what columns still have missing data:

In [53]:
X_train.T[X_train.isna().any()].index

Index(['assists_1_per_games_1', 'attempted_shots_1_per_games_1',
       'goals_1_per_games_1', 'onice_attempts_1_per_games_1',
       'pp_toi_1_per_games_1', 'shots_on_goal_1_per_games_1',
       'toi_1_per_games_1', 'assists_2_per_games_2',
       'attempted_shots_2_per_games_2', 'goals_2_per_games_2',
       'onice_attempts_2_per_games_2', 'pp_toi_2_per_games_2',
       'shots_on_goal_2_per_games_2', 'toi_2_per_games_2',
       'assists_3_per_games_3', 'attempted_shots_3_per_games_3',
       'goals_3_per_games_3', 'onice_attempts_3_per_games_3',
       'pp_toi_3_per_games_3', 'shots_on_goal_3_per_games_3',
       'toi_3_per_games_3', 'assists_4_per_games_4',
       'attempted_shots_4_per_games_4', 'goals_4_per_games_4',
       'onice_attempts_4_per_games_4', 'pp_toi_4_per_games_4',
       'shots_on_goal_4_per_games_4', 'toi_4_per_games_4',
       'assists_5_per_games_5', 'attempted_shots_5_per_games_5',
       'goals_5_per_games_5', 'onice_attempts_5_per_games_5',
       'pp_toi_5_pe

In [57]:
cols_with_na = list(X_train.T[X_train.isna().any()].index)

Let's see how many entries are missing all data from all five windows.

In [67]:
X_train.loc[:, cols_with_na].isna().T.all().sum(), X_test.loc[:, cols_with_na].isna().T.all().sum(), X_lastwindows.loc[:, cols_with_na].isna().T.all().sum()

(331, 52, 48)

These are each less than 5% of the rows in each division, so we can safely drop them.

In [70]:
X_train = X_train.dropna(how='all', subset=cols_with_na)
X_test = X_test.dropna(how='all', subset=cols_with_na)
X_lastwindows = X_lastwindows.dropna(how='all', subset=cols_with_na)

In [71]:
X_train.loc[:, cols_with_na].isna().T.all().sum(), X_test.loc[:, cols_with_na].isna().T.all().sum(), X_lastwindows.loc[:, cols_with_na].isna().T.all().sum()

(0, 0, 0)

In [72]:
X_train.shape[0], X_test.shape[0], X_lastwindows.shape[0]

(11008, 1495, 1254)

Let's find the median values across the league as well as each player's median recent performance in each category, using the average of the two to fill in missing values.

In [74]:
def median_over_window(col):
    train_medians = X_train.groupby('window').agg(median=(col, 'median'))
    last_medians = X_lastwindows.groupby('window').agg(median=(col, 'median'))

    med_train = X_train.merge(train_medians, how='inner', left_index=True, right_index=True)['median']
    med_test = X_test.merge(train_medians, how='inner', left_index=True, right_index=True)['median']
    med_last = X_lastwindows.merge(last_medians, how='inner', left_index=True, right_index=True)['median']

    return med_train, med_test, med_last

In [79]:
stats = ['assists', 'attempted_shots', 'goals', 'onice_attempts', 'pp_toi', 'shots_on_goal', 'toi']

for stat in stats:
    stat_cols = [stat + '_' + str(i) + '_per_games_' + str(i) for i in range(1,6)]
    train_meds = X_train.loc[:,stat_cols].median(axis=1)
    test_meds = X_test.loc[:,stat_cols].median(axis=1)
    last_meds = X_lastwindows.loc[:,stat_cols].median(axis=1)
    for col in stat_cols:
        league_med_train, league_med_test, league_med_last = median_over_window(col)
        fill_train = (train_meds + league_med_train) / 2
        fill_test = (test_meds + league_med_test) / 2
        fill_last = (last_meds + league_med_last) / 2
        X_train[col] = X_train[col].fillna(value=fill_train)
        X_test[col] = X_test[col].fillna(value=fill_test)
        X_lastwindows[col] = X_lastwindows[col].fillna(value=fill_last)

In [81]:
X_train.isna().sum()

isForward                          0
height_cm                          0
weight                             0
age                                0
age_squared                        0
assists_1_per_games_1            278
attempted_shots_1_per_games_1     13
goals_1_per_games_1              524
onice_attempts_1_per_games_1       0
pp_toi_1_per_games_1               0
shots_on_goal_1_per_games_1       25
toi_1_per_games_1                  0
assists_2_per_games_2            278
attempted_shots_2_per_games_2     13
goals_2_per_games_2              524
onice_attempts_2_per_games_2       0
pp_toi_2_per_games_2               0
shots_on_goal_2_per_games_2       25
toi_2_per_games_2                  0
assists_3_per_games_3            278
attempted_shots_3_per_games_3     13
goals_3_per_games_3              524
onice_attempts_3_per_games_3       0
pp_toi_3_per_games_3               0
shots_on_goal_3_per_games_3       25
toi_3_per_games_3                  0
assists_4_per_games_4            278
a

These values look like they are missing because a player had some time played, but did not register any of the given stat in the window. Let's fill these in with 0s.

In [86]:
X_train = X_train.fillna(value=0)
X_test = X_test.fillna(value=0)
X_lastwindows = X_lastwindows.fillna(value=0)

In [88]:
X_train.isna().any().sum(), X_test.isna().any().sum(), X_lastwindows.isna().any().sum()

(0, 0, 0)

We can do the same for the Y values too, since we built X and Y to include only rows where the player had at least 10 games played in the window.

In [100]:
Y_train = Y_train.fillna(value=0)
Y_test = Y_test.fillna(value=0)
Y_lastwindows = Y_lastwindows.fillna(value=0)

## 9 Save the data<a id='9_Save_the_data'></a>

In [101]:
datapath = '../data/processed/'

dataframes = [X_train, Y_train, X_test, Y_test, X_lastwindows, Y_lastwindows]

names = ['X_train', 'Y_train', 'X_test', 'Y_test', 'X_lastwindows', 'Y_lastwindows']

for df, name in zip(dataframes, names):
    save_file(df, name + '.csv', datapath)

Writing file.  "../data/processed/X_train.csv"
Writing file.  "../data/processed/Y_train.csv"
Writing file.  "../data/processed/X_test.csv"
Writing file.  "../data/processed/Y_test.csv"
Writing file.  "../data/processed/X_lastwindows.csv"
Writing file.  "../data/processed/Y_lastwindows.csv"


## 10 Summary<a id='11_Summary'></a>

In this notebook we reorganized our data to be better suited for modeling, created our features, split the training data from test data, and imputed missing values. We will normalize before training our models in the next notebook.